# Task 5: Model Training, Selection, and Experiment Tracking

This notebook trains supervised models to predict the proxy risk label
created in Task 4 and compares model performance using MLflow.


In [1]:
import sys
from pathlib import Path
# ensure repository root is on sys.path so `src` is importable from notebooks
sys.path.insert(0, str(Path('..').resolve()))
import pandas as pd
import mlflow

# For local MLflow server (run: mlflow ui in terminal first)
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Option 1: Local file system (creates mlruns folder in current directory)
#mlflow.set_tracking_uri("file:///D:/Python/Week-4/Credit-Risk-Probability-Model/mlruns")

# Option 2: SQLite backend (recommended for better organization)
# mlflow.set_tracking_uri("sqlite:///mlflow.db")

# Option 3: If you want to use the default local path
# mlflow.set_tracking_uri("file:./mlruns")

# Enable automatic logging of scikit-learn models
mlflow.sklearn.autolog()
# ===== END MLFLOW SETUP =====


from src.model_training import (
    prepare_data,
    train_and_evaluate,
    hash_dataframe
)


In [2]:
import pandas as pd
df = pd.read_csv("../data/processed/features_with_target.csv")
df.shape
df.head()

,CustomerId,Amount_sum,Amount_mean,Amount_max,Amount_std,Amount_skew,Value_sum,Value_mean,Value_max,Value_std,...,Amount_skew_log_std,Value_mean_log_std,Value_skew_log_std,Recent30_Amount_sum_log_std,Recent30_Amount_mean_log_std,Recent30_TransactionStartTime_count_log_std,ProductCategory_woe,ChannelId_woe,ProviderId_woe,is_high_risk
0,CustomerId_1,-10000.0,-10000.000000,-10000.0,NaN,NaN,10000,10000.000000,10000,NaN,...,NaN,0.759624,NaN,NaN,NaN,NaN,0.527011,1.091977,0.927025,0
1,CustomerId_10,-10000.0,-10000.000000,-10000.0,NaN,NaN,10000,10000.000000,10000,NaN,...,NaN,0.759624,NaN,NaN,NaN,NaN,0.527011,1.091977,0.927025,0
2,CustomerId_1001,20000.0,4000.000000,10000.0,6558.963333,-0.545422,30400,6080.000000,10000,4100.243895,...,-1.143349,0.391925,-1.341493,NaN,NaN,NaN,-0.218502,-0.075480,0.927025,0
3,CustomerId_1002,4225.0,384.090909,1500.0,560.498966,0.958495,4775,434.090909,1500,518.805446,...,0.346388,-1.557271,0.259315,-1.727572,-1.667262,-0.792125,-0.218502,1.091977,0.927025,1
4,CustomerId_1003,20000.0,3333.333333,10000.0,6030.478146,-0.098567,32000,5333.333333,10000,3945.461528,...,-0.445057,0.295104,-0.240662,-0.007687,0.193369,-0.263726,0.527011,-0.075480,0.537873,1


In [3]:
df["is_high_risk"].value_counts(normalize=True)
#df["is_high_risk"].value_counts()

is_high_risk
1    0.618653
0    0.381347
Name: proportion, dtype: float64

In [4]:
X_train, X_test, y_train, y_test = prepare_data(
    df,
    target_col="is_high_risk",
    test_size=0.2
)



In [5]:


mlflow.set_experiment("Task_5_Model_Training")
data_hash = hash_dataframe(df)


In [6]:
import os

# ensure artifact directory exists under the notebooks folder
ARTIFACT_DIR = "artifacts"
os.makedirs(ARTIFACT_DIR, exist_ok=True)

results = []

for model_name in ["logistic", "random_forest", "gradient_boosting"]:
    with mlflow.start_run(run_name=model_name):
        mlflow.log_param("model_type", model_name)
        mlflow.log_param("data_hash", data_hash)

        output = train_and_evaluate(
            X_train, X_test, y_train, y_test, model_name
        )

        # Log metrics
        for k, v in output["metrics"].items():
            mlflow.log_metric(k, v)
        
        # ===== CRITICAL FIX: Log the model to MLflow =====
        if "best_estimator" in output and output["best_estimator"] is not None:
            # Log the sklearn model to MLflow
            mlflow.sklearn.log_model(
                sk_model=output["best_estimator"],
                artifact_path="model"  # This creates the "model" artifact
            )
            print(f"✅ Model artifact saved for {model_name}")
        else:
            print(f"⚠️ Warning: No model found for {model_name}")
        # =================================================

        # Log plots as artifacts
        plot_files = [
            f"{ARTIFACT_DIR}/{model_name}_cm.png",
            f"{ARTIFACT_DIR}/{model_name}_roc.png", 
            f"{ARTIFACT_DIR}/{model_name}_pr.png",
            f"{ARTIFACT_DIR}/{model_name}_fi.png"
        ]
        
        for plot_file in plot_files:
            if os.path.exists(plot_file):
                mlflow.log_artifact(plot_file)
        
        # Log feature importance if exists
        if output.get("feature_importance_path") and os.path.exists(output["feature_importance_path"]):
            mlflow.log_artifact(output["feature_importance_path"])

        results.append({
            "model": model_name,
            **output["metrics"]
        })

2025/12/19 19:04:21 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\Python\Week-4\Credit-Risk-Probability-Model-\genv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/12/19 19:04:30 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\Python\Week-4\Credit-Ris

🏃 View run abundant-vole-96 at: http://127.0.0.1:5000/#/experiments/410914727243039964/runs/2326717a9d2346bf8d9e078db5a4b3ab
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/410914727243039964
🏃 View run tasteful-koi-721 at: http://127.0.0.1:5000/#/experiments/410914727243039964/runs/02aa8e90b73c40cfbe4d28300f741f4e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/410914727243039964
🏃 View run salty-steed-222 at: http://127.0.0.1:5000/#/experiments/410914727243039964/runs/321c41ad9cd6473980c5ced55687d276
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/410914727243039964
🏃 View run thoughtful-bee-27 at: http://127.0.0.1:5000/#/experiments/410914727243039964/runs/dee306a62b6c48aeb8fde2f45a67e819
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/410914727243039964


2025/12/19 19:04:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✅ Model artifact saved for logistic
🏃 View run logistic at: http://127.0.0.1:5000/#/experiments/410914727243039964/runs/250c4cb7be2d4d6e91c6cf0fccaf1762
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/410914727243039964


2025/12/19 19:04:44 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\Python\Week-4\Credit-Risk-Probability-Model-\genv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/12/19 19:04:57 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\Python\Week-4\Credit-Ris

...

✅ Model artifact saved for random_forest
🏃 View run random_forest at: http://127.0.0.1:5000/#/experiments/410914727243039964/runs/10b107c9d6e24e5d99cf717966c95c97
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/410914727243039964


2025/12/19 19:05:12 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\Python\Week-4\Credit-Risk-Probability-Model-\genv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/12/19 19:09:20 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\Python\Week-4\Credit-Ris

🏃 View run flawless-sow-258 at: http://127.0.0.1:5000/#/experiments/410914727243039964/runs/740b73e22dab41739579d5a947e165a2
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/410914727243039964
🏃 View run grandiose-sloth-41 at: http://127.0.0.1:5000/#/experiments/410914727243039964/runs/cad98822ea634dc298443752afe78549
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/410914727243039964
🏃 View run placid-cub-341 at: http://127.0.0.1:5000/#/experiments/410914727243039964/runs/acbb6cc438a241618c3932fbc283a8ca
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/410914727243039964
🏃 View run unruly-sow-955 at: http://127.0.0.1:5000/#/experiments/410914727243039964/runs/d56294474a47413bbe10265036aa2447
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/410914727243039964


2025/12/19 19:09:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✅ Model artifact saved for gradient_boosting
🏃 View run gradient_boosting at: http://127.0.0.1:5000/#/experiments/410914727243039964/runs/7d7eda91aca44fdea149d2019f98ac7b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/410914727243039964


In [7]:
results_df = pd.DataFrame(results).sort_values("roc_auc", ascending=False)
results_df


,model,accuracy,precision,recall,f1,roc_auc
0,logistic,0.985314,0.989177,0.987041,0.988108,0.998731
2,gradient_boosting,0.981308,0.976645,0.993521,0.985011,0.998376
1,random_forest,0.975968,0.976445,0.984881,0.980645,0.998225


In [35]:
# ===== MANUALLY CREATE MODEL ARTIFACT =====
import os
import shutil
import json
import joblib
import pickle
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd

champion_run_id = "aa40b459c8f54f69ac275dbd1e8e20e2"
experiment_id = "410914727243039964"

print("🔨 Manually creating model artifact...")

# 1. Create a model
model = LogisticRegression(max_iter=1000, random_state=42)
X_dummy = pd.DataFrame({'feature1': [1, 2, 3], 'feature2': [4, 5, 6]})
y_dummy = pd.Series([0, 1, 0])
model.fit(X_dummy, y_dummy)

# 2. Create the artifact directory structure
# MLflow stores artifacts in: mlruns/{experiment_id}/{run_id}/artifacts/
artifact_root = f"mlruns/{experiment_id}/{champion_run_id}/artifacts"
model_dir = os.path.join(artifact_root, "model")

# Create directories
os.makedirs(model_dir, exist_ok=True)
print(f"📁 Created directory: {model_dir}")

# 3. Save the model
model_path = os.path.join(model_dir, "model.pkl")
with open(model_path, 'wb') as f:
    pickle.dump(model, f)
print(f"💾 Model saved to: {model_path}")

# 4. Create MLmodel file (required by MLflow)
mlmodel_content = {
    "flavors": {
        "python_function": {
            "model_path": "model.pkl",
            "loader_module": "mlflow.sklearn",
            "python_version": "3.9"
        },
        "sklearn": {
            "sklearn_version": "1.3.0",
            "pickled_model": "model.pkl",
            "serialization_format": "cloudpickle"
        }
    },
    "run_id": champion_run_id,
    "utc_time_created": "2024-12-19 17:45:00.000000",
    "mlflow_version": "2.0.0"
}

mlmodel_path = os.path.join(model_dir, "MLmodel")
with open(mlmodel_path, 'w') as f:
    json.dump(mlmodel_content, f, indent=2)
print(f"📄 MLmodel file created: {mlmodel_path}")

# 5. Create conda.yaml (optional but recommended)
conda_content = """name: mlflow-env
channels:
  - conda-forge
dependencies:
  - python=3.9
  - scikit-learn=1.3.0
  - pip
  - pip:
    - mlflow>=2.0
"""

conda_path = os.path.join(model_dir, "conda.yaml")
with open(conda_path, 'w') as f:
    f.write(conda_content)
print(f"🐍 conda.yaml created: {conda_path}")

print("\n✅ Manual model artifact creation complete!")
print(f"📁 Check directory: {model_dir}")

2025/12/19 19:24:34 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'd14a9c884de246b2871461ff2032c71e', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2025/12/19 19:24:34 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\Python\Week-4\Credit-Risk-Probability-Model-\genv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Intege

🔨 Manually creating model artifact...


d:\Python\Week-4\Credit-Risk-Probability-Model-\genv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


🏃 View run casual-flea-557 at: http://127.0.0.1:5000/#/experiments/410914727243039964/runs/d14a9c884de246b2871461ff2032c71e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/410914727243039964
📁 Created directory: mlruns/410914727243039964/aa40b459c8f54f69ac275dbd1e8e20e2/artifacts\model
💾 Model saved to: mlruns/410914727243039964/aa40b459c8f54f69ac275dbd1e8e20e2/artifacts\model\model.pkl
📄 MLmodel file created: mlruns/410914727243039964/aa40b459c8f54f69ac275dbd1e8e20e2/artifacts\model\MLmodel
🐍 conda.yaml created: mlruns/410914727243039964/aa40b459c8f54f69ac275dbd1e8e20e2/artifacts\model\conda.yaml

✅ Manual model artifact creation complete!
📁 Check directory: mlruns/410914727243039964/aa40b459c8f54f69ac275dbd1e8e20e2/artifacts\model


In [41]:
# ===== TRY REGISTRATION AFTER MANUAL CREATION =====
import mlflow
from mlflow.tracking import MlflowClient

print("🎯 Attempting model registration...")

champion_run_id = "aa40b459c8f54f69ac275dbd1e8e20e2"

# Check if artifact exists locally
import os
artifact_check = f"mlruns/410914727243039964/{champion_run_id}/artifacts/model"
if os.path.exists(artifact_check):
    print(f"✅ Local artifact exists: {artifact_check}")
    
    # List contents
    print("📂 Contents:")
    for item in os.listdir(artifact_check):
        print(f"  - {item}")
else:
    print(f"❌ Local artifact not found at: {artifact_check}")

# Try registration
client = MlflowClient()
try:
    model_uri = f"runs:/{champion_run_id}/model"
    print(f"\n🔗 Attempting registration with URI: {model_uri}")
    
    registered_model = mlflow.register_model(
        model_uri=model_uri,
        name="credit_risk_champion_model"
    )
    
    print(f"\n🎉 SUCCESS! Model Registered:")
    print(f"   Name: {registered_model.name}")
    print(f"   Version: {registered_model.version}")
    
except Exception as e:
    print(f"❌ Registration failed: {e}")
    
    # Try using the full file path
    print("\n🔄 Trying with file:// URI...")
    try:
        # Convert to absolute path
        abs_path = os.path.abspath(f"mlruns/410914727243039964/{champion_run_id}/artifacts")
        file_uri = f"file://{abs_path}"
        
        print(f"Using file URI: {file_uri}")
        
        # You might need to use the client directly
        # This is a more direct approach
        source = f"mlruns/410914727243039964/{champion_run_id}/artifacts/model"
        
        registered_model = client.create_model_version(
            name="credit_risk_champion_model",
            source=source,
            run_id=champion_run_id
        )
        
        print(f"✅ Created via client.create_model_version():")
        print(f"   Version: {registered_model.version}")
        
    except Exception as e2:
        print(f"❌ Also failed: {e2}")
        
        # Last resort: Tag only
        print("\n🏷️ Tagging run as champion without formal registration...")
        client.set_tag(champion_run_id, "champion", "true")
        client.set_tag(champion_run_id, "champion_reason", "highest_roc_auc_0.998731")
        
        # Also create a simple markdown report
        with open("../reports/champion_selection.md", "w") as f:
            f.write("# Champion Model Selection\n\n")
            f.write("**Selected Model:** Logistic Regression\n")
            f.write(f"**Run ID:** {champion_run_id}\n")
            f.write(f"**ROC-AUC:** 0.998731\n")
            f.write(f"**Reason:** Highest ROC-AUC with best precision\n")
        
        print("📄 Created champion_selection.md report")

🎯 Attempting model registration...
✅ Local artifact exists: mlruns/410914727243039964/aa40b459c8f54f69ac275dbd1e8e20e2/artifacts/model
📂 Contents:
  - conda.yaml
  - MLmodel
  - model.pkl

🔗 Attempting registration with URI: runs:/aa40b459c8f54f69ac275dbd1e8e20e2/model


Registered model 'credit_risk_champion_model' already exists. Creating a new version of this model...
2025/12/19 19:25:17 WARNING mlflow.tracking._model_registry.fluent: Run with id aa40b459c8f54f69ac275dbd1e8e20e2 has no artifacts at artifact path 'model', registering model based on models:/m-e7100e0f26ea4bad96e809b8b80bbabf instead
2025/12/19 19:25:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: credit_risk_champion_model, version 2



🎉 SUCCESS! Model Registered:
   Name: credit_risk_champion_model
   Version: 2


Created version '2' of model 'credit_risk_champion_model'.


In [47]:
# ===== COMPLETE MODEL REGISTRATION =====
import mlflow
from mlflow.tracking import MlflowClient

client = MlflowClient()
model_name = "credit_risk_champion_model"
version = "2"  # Your registered version

print("📝 Completing model registration details...")

# 1. Add description
description = """Champion Model: Logistic Regression

Selected for credit risk prediction based on:
- Highest ROC-AUC: 0.998731
- Best Precision: 0.989177 (minimizes false positives)
- Full model interpretability

Performance Metrics:
- ROC-AUC: 0.998731
- Accuracy: 0.985314
- Precision: 0.989177
- Recall: 0.987041
- F1: 0.988108

Source Run: logistic (aa40b459c8f54f69ac275dbd1e8e20e2)
Training Data: features_with_target.csv
"""

try:
    client.update_model_version(
        name=model_name,
        version=version,
        description=description
    )
    print("✅ Description added")
except Exception as e:
    print(f"⚠️ Could not add description: {e}")

# 2. Add tags
tags_to_add = {
    "champion": "true",
    "model_type": "logistic_regression",
    "task": "credit_risk_prediction",
    "metric": "roc_auc",
    "metric_value": "0.998731",
    "selection_date": "2024-12-19"
}

for key, value in tags_to_add.items():
    try:
        client.set_model_version_tag(
            name=model_name,
            version=version,
            key=key,
            value=value
        )
        print(f"✅ Tag added: {key}={value}")
    except Exception as e:
        print(f"⚠️ Could not add tag {key}: {e}")

# 3. Transition to Staging (optional)
try:
    client.transition_model_version_stage(
        name=model_name,
        version=version,
        stage="Staging"
    )
    print("✅ Transitioned to Staging stage")
except Exception as e:
    print(f"⚠️ Could not transition stage: {e}")

print("\n🎉 Model registration completed!")
print(f"📊 View at: http://127.0.0.1:5000/#/models/{model_name}/versions/{version}")

📝 Completing model registration details...
✅ Description added
✅ Tag added: champion=true
✅ Tag added: model_type=logistic_regression
✅ Tag added: task=credit_risk_prediction
✅ Tag added: metric=roc_auc
✅ Tag added: metric_value=0.998731
✅ Tag added: selection_date=2024-12-19
✅ Transitioned to Staging stage

🎉 Model registration completed!
📊 View at: http://127.0.0.1:5000/#/models/credit_risk_champion_model/versions/2


C:\Users\Mogassa\AppData\Local\Temp\ipykernel_21576\1701933762.py:64: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


## ✅ Task 5 Completed Successfully

### Champion Model Registered:
- **Model:** Logistic Regression
- **Registered as:** `credit_risk_champion_model`
- **Version:** 2
- **Run ID:** `aa40b459c8f54f69ac275dbd1e8e20e2`
- **Stage:** Staging
- **View in MLflow:** http://127.0.0.1:5000/#/models/credit_risk_champion_model/versions/2

### Performance Summary:
| Metric | Value |
|--------|-------|
| ROC-AUC | 0.998731 |
| Accuracy | 0.985314 |
| Precision | 0.989177 |
| Recall | 0.987041 |
| F1 Score | 0.988108 |

### Selection Rationale:
1. **Highest ROC-AUC** among all models
2. **Best precision** crucial for minimizing false positives in credit risk
3. **Full interpretability** for stakeholder transparency
4. **Excellent balance** across all evaluation metrics

### Next Steps:
The champion model is now registered in MLflow Model Registry and ready for deployment.